<a href="https://www.kaggle.com/code/jamesdavey/20x-faster-pixel-coordinate-generator-3s-runtime?scriptVersionId=122514442" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Pixel coordinate generation

In this notebook, I have rewritten the process for getting pixels coordinates described in the [Tutorial notebook](https://www.kaggle.com/code/jpposma/vesuvius-challenge-ink-detection-tutorial) 

The aim of it is to assist the process of dataset creation described in that notebook, and to rapidly speed it up.

The original way took ~70 seconds, my way speeds this up by eliminating the need for a loop, allowing it to run in ~3 seconds. Over 20x faster :)

At the bottom of the notebook I verify both ways produce identical outputs so we can be sure it is working correctly.

In [ ]:
import torch
import numpy as np
import PIL
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import os
import time

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"on device: {device}")

dist_from_centre = 30 # ie. each square into network is 30*2+1=61 wide
evaluation_rectangle = (1100, 3500, 700, 950) # (x, y, w, h)

In [ ]:
root_filepath = '/kaggle/input/vesuvius-challenge-ink-detection/train/1/'
#['inklabels.png', 'inklabels_rle.csv', 'ir.png', 'mask.png', 'surface_volume']
tif_filenames = sorted(os.listdir(root_filepath+'surface_volume'))
print(len(tif_filenames), tif_filenames[:5])

mask = torch.from_numpy(np.array(PIL.Image.open(root_filepath+"mask.png").convert('1')))
label = torch.from_numpy(np.array(PIL.Image.open(root_filepath+"inklabels.png"))).float().to(device)

The functions below produce all pixel coordinates for pixels within the rectangle and outside it. (Note: In both coordinates are restricted to the mask)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(label.cpu(), cmap = 'gray')
patch1 = matplotlib.patches.Rectangle((evaluation_rectangle[0], evaluation_rectangle[1]), evaluation_rectangle[2], evaluation_rectangle[3], linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(patch1)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(mask, cmap = 'gray')
patch1 = matplotlib.patches.Rectangle((evaluation_rectangle[0], evaluation_rectangle[1]), evaluation_rectangle[2], evaluation_rectangle[3], linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(patch1)
plt.show()

In [ ]:
def get_pixels():
    """
    Takes in a mask, and outputs the set of pixels (x,y) in intersection of that mask with a rectangle
    """
    t1 = time.time()
    h, w = mask.shape # 8181, 6330
    full_square = torch.full((h, w, 2), 0, dtype = torch.int)
    for x in range(w): full_square[:, x, 1] = x # (x, y)
    for y in range(h): full_square[y, :, 0] = y
    valid_pixels = full_square.masked_fill(mask.unsqueeze(dim=2) == 0, -1)
    #Cannot crop border as it will ruin our indexing. Instead we set its values to -1 (i.e. outside the mask)
    # valid_pixels = valid_pixels[dist_from_centre:-dist_from_centre, dist_from_centre:-dist_from_centre]
    valid_pixels[:dist_from_centre, :, 0] = -1
    valid_pixels[-dist_from_centre:, :, 0] = -1
    valid_pixels[:, :dist_from_centre, 0] = -1
    valid_pixels[:, -dist_from_centre:, 0] = -1
    #torch.Size([8181, 6330, 2]) --> torch.Size([8121, 6270, 2])
    
    x, y, w, h = evaluation_rectangle
    pixels_inside_rect = valid_pixels[y:y+h+1, x:x+w+1].flatten(end_dim=1) 
    pixels_inside_rect = pixels_inside_rect[pixels_inside_rect[:, 0] != -1].numpy() # apply mask
    
    valid_pixels[y:y+h+1, x:x+w+1, 0] = -1
    flattened = valid_pixels.flatten(end_dim=1)
    pixels_outside_rect = flattened[flattened[:, 0] != -1].type(torch.int).numpy()
    
    print(f"Time taken: {time.time()-t1:.2f} seconds | pixels_inside_rect: {pixels_inside_rect.shape[0]} | pixels_outside_rect: {pixels_outside_rect.shape[0]}")
    return pixels_inside_rect, pixels_outside_rect

pixels_inside_rect2, pixels_outside_rect2 = get_pixels()

In [ ]:
BUFFER = dist_from_centre
rect = evaluation_rectangle

t1 = time.time()
print("Generating pixel lists...")
# Split our dataset into train and val. The pixels inside the rect are the 
# val set, and the pixels outside the rect are the train set.
pixels_inside_rect = []
pixels_outside_rect = []
for pixel in zip(*np.where(mask == 1)):
    if pixel[1] < BUFFER or pixel[1] >= mask.shape[1]-BUFFER or pixel[0] < BUFFER or pixel[0] >= mask.shape[0]-BUFFER:
        continue # Too close to the edge
    if pixel[1] >= rect[0] and pixel[1] <= rect[0]+rect[2] and pixel[0] >= rect[1] and pixel[0] <= rect[1]+rect[3]:
        pixels_inside_rect.append(pixel)
    else:
        pixels_outside_rect.append(pixel)

print(f"Time taken: {time.time()-t1:.2f} seconds | pixels_inside_rect: {len(pixels_inside_rect)} | pixels_outside_rect: {len(pixels_outside_rect)}")

In [ ]:
pixels_inside_rect2, np.array(pixels_inside_rect)

In [ ]:
#Pixel coordinates match up perfectly :)

print((np.array(pixels_inside_rect) == pixels_inside_rect2).min())
print((np.array(pixels_outside_rect) == pixels_outside_rect2).min())